In [ ]:
%pylab inline
from numba import jit

In [ ]:
def fake_root_finder(func, a, b, arg):
    return (func(a, *arg) + func(b, *arg)) / 2

def root(t, u, v):
    func = t**3*(np.sin(t))**2+u*v
    return func

def find_root_loop(iter):
    for i in range(iter):
        r = np.random.random()
        fake_root_finder(root, -2.2, 5, arg=(r, 1))

In [ ]:
%time find_root_loop(1000000)

In [ ]:
@jit(nopython=True)
def fake_root_finder_jited(func, a, b, arg):
    return (func(a, *arg) + func(b, *arg)) / 2

@jit(nopython=True)
def root(t, u, v):
    func = t**3*(np.sin(t))**2+u*v
    return func

@jit(nopython=True)
def find_root_loop_jited(iter):
    for i in range(iter):
        r = np.random.random()
        fake_root_finder_jited(root, -2.2, 5, arg=(r, 1))

In [ ]:
%%time 
find_root_loop_jited(1000000)

In [ ]:
%time find_root_loop(1000000)
%time find_root_loop_jited(1000000)

In [ ]:
import numpy as np
import numba as nb
from numba import jit

def bisection(func, a, b, tol=1e-12, iter_nmax=100):
    fa = func(a)
    fb = func(b)
    if fa*fb >= 0: 
        raise ValueError("The root is not bracketed since f(a) and f(b) have the same sign.")
    if np.abs(fa)<np.abs(fb):
        temp = a
        a = b
        b = temp

    iter_n = 0
    while iter_n < iter_nmax:
        c = (a + b) / 2
        fc = func(c)
        if func(c) == 0 or (b-a)/2 < tol:
            # root found
            return c
        iter_n += 1

        if fc*fa:
            a = c
        else:
            b = c
    else:
        raise Exception("Root non-found. Exceeded max. number of iteration.")

bisection_jited = jit(nopython=True)(bisection)

In [ ]:
from time import time
def root(t):
    func = t**3*(np.sin(t))**2+1
    return func
root_jited = jit(nopython=True)(root)

loop_num = 100000
def loop_bisection(arg):
    result = 0
    for i in range(loop_num):
        rn = np.random.random()
        result += arg*rn*bisection(root, -2.2, 5)
@jit(nopython=True, parallel=True)
def loop_bisection_jited(arg):
    result = 0
    for i in nb.prange(loop_num):
        rn = np.random.random()
        result += arg*rn*bisection_jited(root_jited, -2.2, 5)

In [ ]:
%%time
loop_bisection(10)


In [ ]:
%%time
loop_bisection_jited(89)

In [ ]:
import numpy as np
import numba as nb
from numba import jit
from root_finder import bisection_arg, brentq

@jit(nopython=True, fastmath=True)
def root(t, u, v) -> float:
    func = t**3*(np.sin(t))**2 +u*v
    return func

def find_root_loop(iter):
    result = 0
    for i in nb.prange(iter):
        r = np.random.random()
        result += bisection_arg(root, -2.2, 5, (1.0, 1.0))
    return result/iter

find_root_loop_jited = jit(nopython=True, parallel=True, fastmath=True)(find_root_loop)
%time find_root_loop(300000)
%time find_root_loop_jited(300000)


In [ ]:
from numba import jit
import numba as nb
from tele_geo_jit import z1, m1_into_tele
z1_jit = jit(nopython=True, parallel=True, fastmath=True)(z1)
m1_into_tele_jit =  jit(nopython=True, fastmath=True)(m1_into_tele)

def z1_loop(iter):
    for i in nb.prange(iter):
        x = np.random.random()
        y = np.random.random()
        z1(x, y)
    return 0

@jit(nopython=True, parallel=True, fastmath=True)
def z1_loop_jit(iter):
    for i in nb.prange(iter):
        x = np.random.random()
        y = np.random.random()
        z1_jit(x, y)
    return 0

def m1_into_tele_loop(iter):
    for i in nb.prange(iter):
        x = np.random.random()
        y = np.random.random()
        z = np.random.random()
        m1_into_tele(x, y, z)
    return 0

@jit(nopython=True, parallel=True, fastmath=True)
def m1_into_tele_loop_jit(iter):
    for i in nb.prange(iter):
        x = np.random.random()
        y = np.random.random()
        z = np.random.random()
        m1_into_tele_jit(x, y, z)
    return 0


In [ ]:
%time z1_loop(20000)
%time m1_into_tele_loop(10000)

In [ ]:
%time z1_loop_jit(1)
%time m1_into_tele_loop_jit(1)

%time z1_loop_jit(20000)
%time m1_into_tele_loop_jit(10000)

In [13]:
import numpy as np
import numba as nb
from numba import jit

# @jit(nopython=True, parallel=True, fastmath=True)
def generate_theta_phi_array(theta_a, theta_b, theta_N, phi_a, phi_b, phi_N):
    theta = np.linspace(theta_a, theta_b, theta_N)
    phi = np.linspace(phi_a, phi_b, phi_N)
    theta = theta.repeat(theta_N).reshape((theta_N, theta_N)).T.flatten()
    phi = phi.repeat(phi_N)
    return theta, phi
generate_theta_phi_array_jit = jit(nopython=True, parallel=True, fastmath=True)(generate_theta_phi_array)

theta_a, theta_b, theta_N = 0, 4, 4
phi_a, phi_b, phi_N = 0.1, 3.1, 4
theta, phi = generate_theta_phi_array(theta_a, theta_b, theta_N, phi_a, phi_b, phi_N)
theta, phi = generate_theta_phi_array_jit(theta_a, theta_b, theta_N, phi_a, phi_b, phi_N)


theta_orig = np.linspace(theta_a, theta_b, theta_N)
phi_orig = np.linspace(phi_a, phi_b, phi_N)
theta_orig, phi_orig = np.meshgrid(theta_orig, phi_orig)
theta_orig = np.ravel(theta_orig)
phi_orig = np.ravel(phi_orig)

assert theta_orig.all() == theta.all()
assert phi_orig.all() == phi.all()
print(theta_orig)
print(theta)
print(phi_orig)
print(phi)

theta_a, theta_b, theta_N = 0, 4, 500
phi_a, phi_b, phi_N = 0.1, 3.1, 500
%time theta, phi = generate_theta_phi_array(theta_a, theta_b, theta_N, phi_a, phi_b, phi_N)
%time theta, phi = generate_theta_phi_array_jit(theta_a, theta_b, theta_N, phi_a, phi_b, phi_N)

[0.         1.33333333 2.66666667 4.         0.         1.33333333
 2.66666667 4.         0.         1.33333333 2.66666667 4.
 0.         1.33333333 2.66666667 4.        ]
[0.         1.33333333 2.66666667 4.         0.         1.33333333
 2.66666667 4.         0.         1.33333333 2.66666667 4.
 0.         1.33333333 2.66666667 4.        ]
[0.1 0.1 0.1 0.1 1.1 1.1 1.1 1.1 2.1 2.1 2.1 2.1 3.1 3.1 3.1 3.1]
[0.1 0.1 0.1 0.1 1.1 1.1 1.1 1.1 2.1 2.1 2.1 2.1 3.1 3.1 3.1 3.1]
Wall time: 5 ms
Wall time: 4 ms


In [6]:
import numpy as np
import numba as nb
from numba import jit
from tele_geo_jit import *

@jit(nopython=True, fastmath=True)
def root_z2(t, x_0, y_0, z_0, alpha, beta, gamma):
    # Endpoint of ray:
    x = x_0 + alpha * t
    y = y_0 + beta * t
    z = z_0 + gamma * t

    # Convert to M2 r.f.
    xm2, ym2, zm2 = tele_into_m2(x, y, z)

    # Z of mirror in M2 r.f.
    z_m2 = z2(xm2, ym2)
    return zm2 - z_m2
t = 133.0
x_0, y_0, z_0, alpha, beta, gamma = 0, 0, 0, 1.2, 0.5, 0.1
result = root_z2(t, x_0, y_0, z_0, alpha, beta, gamma)

In [18]:
import numpy as np
diff = np.array([0.4e-11, 0.1e-11, 0.11e-11])
if diff.any() < 1e-11:
    print("OH Yes!")
else:
    print("nonono")

nonono
